In [ ]:
import os

In [ ]:
%run "/home/jan/Documentos/tcc/dependencias/conn_spark.py"

spark = cria_conn("gold")

In [ ]:
path_base = '/home/jan/Documentos/tcc/datasets/prata/CONSUMO_ENERGIA'
arquivos = os.listdir(path_base)

for arquivo in arquivos:
    df = spark.read.parquet(f'{path_base}/{arquivo}')
    df.createOrReplaceTempView(f'{arquivo.replace(".parquet","")}')

    print(f"Criado view temporária do arquivo: {arquivo}")

In [ ]:
case_competencia = """CONCAT(t1.ano, '-', 
         CASE 
           WHEN t1.mes = 'JAN' THEN '01'
           WHEN t1.mes = 'FEV' THEN '02'
           WHEN t1.mes = 'MAR' THEN '03'
           WHEN t1.mes = 'ABR' THEN '04'
           WHEN t1.mes = 'MAI' THEN '05'
           WHEN t1.mes = 'JUN' THEN '06'
           WHEN t1.mes = 'JUL' THEN '07'
           WHEN t1.mes = 'AGO' THEN '08'
           WHEN t1.mes = 'SET' THEN '09'
           WHEN t1.mes = 'OUT' THEN '10'
           WHEN t1.mes = 'NOV' THEN '11'
           WHEN t1.mes = 'DEZ' THEN '12'
         END, '-01')"""

In [ ]:
fato = spark.sql(f"""
        SELECT 
                {case_competencia} AS dt_Competencia, 
                CAST(t1.consumo_total AS INTEGER) AS total_consumoMWh,
                CAST(t2.consumo_total AS INTEGER) AS total_consumoCativoMWh,
                CAST(t3.consumo_total AS INTEGER) AS total_consumoLivreMWh,
                CAST(t4.consumo_total AS INTEGER) AS total_consumoResidencialMWh,
                CAST(t5.consumo_total AS INTEGER) AS total_consumoIndustrialMWh,
                CAST(t6.consumo_total AS INTEGER) AS total_consumoComercialMWh,
                CAST(t7.consumo_total AS INTEGER) AS total_consumoOutrosMWh,
                CAST(t8.consumo_total AS INTEGER) AS total_consumidoresResidencialMWh,
                CAST(t9.consumo_total AS INTEGER) AS total_consumidoresIndustriaisMWh,
                CAST(t10.consumo_total AS INTEGER) AS total_consumidoresComerciaisMWh,
                CAST(t11.consumo_total AS INTEGER) AS total_consumidoresOutrosMWh,
                CAST(t12.consumo_total AS INTEGER) AS total_consumidoresCativosMWh,
                CAST(t13.consumo_total AS INTEGER) AS total_consumidoresLivresMWh      
        FROM CONSUMO_POR_UF t1
        LEFT JOIN CONSUMO_CATIVO_POR_UF t2 ON {case_competencia.replace('t1', 't2')} = {case_competencia}
        LEFT JOIN CONSUMO_LIVRE_POR_UF t3 ON {case_competencia.replace('t1', 't3')} = {case_competencia}
        LEFT JOIN CONSUMO_RESIDENCIAL_POR_UF t4 ON {case_competencia.replace('t1', 't4')} = {case_competencia}
        LEFT JOIN CONSUMO_INDUSTRIAL_POR_UF t5 ON {case_competencia.replace('t1', 't5')} = {case_competencia}
        LEFT JOIN CONSUMO_COMERCIAL_POR_UF t6 ON {case_competencia.replace('t1', 't6')} = {case_competencia}
        LEFT JOIN CONSUMO_OUTROS_POR_UF t7 ON {case_competencia.replace('t1', 't7')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_RES_POR_UF t8 ON {case_competencia.replace('t1', 't8')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_INDUSTRIAIS_POR_UF t9 ON {case_competencia.replace('t1', 't9')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_COMERCIAIS_POR_UF t10 ON {case_competencia.replace('t1', 't10')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_OUTROS_POR_UF t11 ON {case_competencia.replace('t1', 't11')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_CATIVOS_POR_UF t12 ON {case_competencia.replace('t1', 't12')} = {case_competencia}
        LEFT JOIN CONSUMIDORES_LIVRES_POR_UF t13 ON {case_competencia.replace('t1', 't13')} = {case_competencia}
        WHERE {case_competencia} IS NOT NULL AND YEAR({case_competencia}) < 2025
        order by dt_competencia asc
""")

fato.show()

In [ ]:
path_gold = '/home/jan/Documentos/tcc/datasets/gold'

os.makedirs(path_gold, exist_ok=True)

fato.toPandas().to_csv(f'{path_gold}/CONSUMO_ENERGIA.csv')

In [ ]:
spark.stop()